## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation - *sur la glace ; ça pourrait permettre d'éviter les faux négatifs dûs à des variations singulier/pluriel quand on essaie d'extraire les termes qui existent déjà dans la taxonomie*)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH présents dans les données

### **Lire le corpus** 

In [1]:
lng = 'fr'
acteur = 'chum'
tag = ''
if tag:
    file = acteur + '/' + acteur + '_' + tag + '.csv'

else:
    if lng == 'fr':
        file = acteur +'.csv'
    if lng == 'en':
        file = acteur + '_en.csv'

In [2]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [3]:
def lire_corpus(acteur = acteur, langue=lng):
    base_path = '../03-corpus/2-data/'
    
    folder_path = path.join(base_path, '1-' + langue, acteur)
    all_files = glob.glob(path.join(folder_path, "*.csv"))
    tags = [f.split('_')[1][:-4] for f in listdir(folder_path)]

    df = DataFrame()
    for f, tag in zip(all_files, tags):
        csv = read_csv(f, encoding='utf-8', sep=',')
        csv = csv[~csv["Address"].str.contains('pdf')] #  Problèmes 
        #csv = csv['text']
        # Using DataFrame.insert() to add a column
        df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'text']]
        #df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'Type', 'text']]
    return df

In [4]:
data = lire_corpus(acteur) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 1605 documents.


In [5]:
base_path = '../03-corpus/2-data/'
folder_path = path.join(base_path, '1-' + lng, acteur)

data.to_csv(folder_path + '.csv')
data

,Corpus,Sous-corpus,Title,text
0,chum,Cancer du sein,5 octobre 2022 - Soirée BRAVOURE sur le cancer...,5 octobre 2022 - Soirée BRAVOURE sur le cancer...
1,chum,Cancer du sein,Cancer du sein,Cancer du sein Le cancer du sein touche 1 femm...
2,chum,Cancer du sein,Reconstruction mammaire,Reconstruction mammaire Toutes les femmes ayan...
3,chum,Cancer du sein,Cancer métastatique,Cancer métastatique Le cancer du sein métastat...
4,chum,Cancer du sein,Événements,"Événements À titre de centre universitaire, le..."
...,...,...,...,...
30,chum,Services,Phénotypage de rongeurs - Métabolique,Phénotypage de rongeurs - Métabolique La plate...
31,chum,Services,Physiologie cellulaire,"Physiologie cellulaire Depuis 2013, la platefo..."
32,chum,Services,Radiochimie et cyclotron,Radiochimie et cyclotron Notre équipe est spéc...
33,chum,Services,Transgénèse et modélisation animale,Transgénèse et modélisation animale La platefo...


### **Nettoyage**

In [6]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
# phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['text'].tolist()]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [7]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 1605 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [8]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [9]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [10]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

temps = tok(corpus)

Avec le RegExpTokenizer, notre corpus contient 433868 tokens.
Le POS tagging devrait prendre environ 0 minutes.


In [11]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append([t.split('\t')[0], t.split('\t')[1]])
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output


In [12]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

In [13]:
tagged

[['une', 'DET:ART'],
 ['équipe', 'NOM'],
 ['franco-québécoise', 'ADJ'],
 ['identifie', 'VER:pres'],
 ['le', 'DET:ART'],
 ['gène', 'NOM'],
 ['responsable', 'ADJ'],
 ["d'", 'PRP'],
 ['une', 'DET:ART'],
 ['forme', 'NOM'],
 ['inusitée', 'ADJ'],
 ['du', 'PRP:det'],
 ['syndrome', 'NOM'],
 ['de', 'PRP'],
 ['cushing', 'NOM'],
 ['drs', 'ADJ'],
 ['andré', 'ADJ'],
 ['lacroix', 'NOM'],
 ['et', 'KON'],
 ['isabelle', 'ADJ'],
 ['bourdeau', 'NOM'],
 ['15', 'NUM'],
 ['octobre', 'NOM'],
 ['2021', 'NUM'],
 ['en', 'PRP'],
 ['5', 'NUM'],
 ['secondes', 'NOM'],
 ['des', 'PRP:det'],
 ['scientifiques', 'ADJ'],
 ['montréalais', 'NOM'],
 ['et', 'KON'],
 ['parisiens', 'NOM'],
 ['ont', 'VER:pres'],
 ['découvert', 'VER:pper'],
 ['les', 'DET:ART'],
 ['causes', 'NOM'],
 ['moléculaires', 'ADJ'],
 ['derrière', 'PRP'],
 ['une', 'DET:ART'],
 ['forme', 'NOM'],
 ['particulière', 'ADJ'],
 ['du', 'PRP:det'],
 ['syndrome', 'NOM'],
 ['de', 'PRP'],
 ['cushing', 'NOM'],
 ['induite', 'VER:pper'],
 ['par', 'PRP'],
 ["l'", 'DET:ART

### **Lemmatisation** (FrenchLefffLemmatizer)
Sur pause ; peut-être qu'il vaudrait mieux le faire plus tard (au moment d'identifier si chaque terme existe déjà dans la taxo ou non) afin d'éviter de complexifier toutes les autres étapes

In [14]:
# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
# lemmatizer = FrenchLefffLemmatizer()

#### **Mapping POS Tags** (FRMG)
On doit mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)
#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [15]:
# file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

# with open(file_path) as f:
#    csv = read_csv(f)

# treeTag = [term for term in csv['TreeTagger'].tolist()] 
# lefff = [term for term in csv['Lefff'].tolist()]

# mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [16]:
# df_tagged = DataFrame(tagged, columns=['Token', 'Tag TreeTagger'])
# df_tagged['Tag Lefff'] = df_tagged['Tag TreeTagger'].map(mapping)
# df_tagged['Lemme'] = df_tagged['Token'] # S'il est pas capable de lemmatiser ensuite, on garde l'expression telle quelle

In [17]:
# tagged_dict = df_tagged.to_dict('records')

In [18]:
# # tokens = df_tagged['Token'].tolist()
# # tags = df_tagged['Tag Lefff'].tolist()
# # tagged_lefff = [(t, tg) for t,tg in zip(tokens, tags)]

# for term in tagged_dict:
#     if lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff']) == []:
#         term_l = lemmatizer.lemmatize(word=term['Token'])
    
#     elif type(lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])) == str:
#         term_l  = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])

#     else:
#         term_l = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])[0][0]

#     term['Lemme'] = term_l

In [19]:
# tagged_dict

In [20]:
# tagged = DataFrame(tagged_dict)
# tagged = tagged.drop(columns=['Tag Lefff']).values.tolist()

# tagged

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [21]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=2, max_len=8))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [22]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 3121489 ngrammes.


In [23]:
ngrammes

[(['une', 'DET:ART'], ['équipe', 'NOM']),
 (['une', 'DET:ART'], ['équipe', 'NOM'], ['franco-québécoise', 'ADJ']),
 (['une', 'DET:ART'],
  ['équipe', 'NOM'],
  ['franco-québécoise', 'ADJ'],
  ['identifie', 'VER:pres']),
 (['une', 'DET:ART'],
  ['équipe', 'NOM'],
  ['franco-québécoise', 'ADJ'],
  ['identifie', 'VER:pres'],
  ['le', 'DET:ART']),
 (['une', 'DET:ART'],
  ['équipe', 'NOM'],
  ['franco-québécoise', 'ADJ'],
  ['identifie', 'VER:pres'],
  ['le', 'DET:ART'],
  ['gène', 'NOM']),
 (['une', 'DET:ART'],
  ['équipe', 'NOM'],
  ['franco-québécoise', 'ADJ'],
  ['identifie', 'VER:pres'],
  ['le', 'DET:ART'],
  ['gène', 'NOM'],
  ['responsable', 'ADJ']),
 (['une', 'DET:ART'],
  ['équipe', 'NOM'],
  ['franco-québécoise', 'ADJ'],
  ['identifie', 'VER:pres'],
  ['le', 'DET:ART'],
  ['gène', 'NOM'],
  ['responsable', 'ADJ'],
  ["d'", 'PRP']),
 (['équipe', 'NOM'], ['franco-québécoise', 'ADJ']),
 (['équipe', 'NOM'], ['franco-québécoise', 'ADJ'], ['identifie', 'VER:pres']),
 (['équipe', 'NOM'],

### **Extraction des patrons syntaxiques**

In [24]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [25]:
phrases = extract_patterns(ngrammes)
# phrases_lem = extract_patterns(ngrammes_lem)

In [26]:
# def freq(phrases):
#     return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

# def freq(phrases):
#     return FreqDist([t[0] for t in phrases])

In [27]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=10))

In [28]:
frequencies

FreqDist({('STOP',): 33147, ('de',): 25273, ('.',): 13057, ('et',): 9465, ('la',): 9425, ('le',): 9207, ('du',): 7517, ("l'",): 7145, ("d'",): 7080, ('des',): 6929, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [29]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [30]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [31]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [32]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

phrases = filter_len(phrases)

In [33]:
phrases

[[('gène', 'responsable'), ['NOM', 'ADJ']],
 [('gène', 'responsable', "d'", 'une', 'forme'),
  ['NOM', 'ADJ', 'PRP', 'DET:ART', 'NOM']],
 [('gène', 'responsable', "d'", 'une', 'forme', 'inusitée'),
  ['NOM', 'ADJ', 'PRP', 'DET:ART', 'NOM', 'ADJ']],
 [('gène', 'responsable', "d'", 'une', 'forme', 'inusitée', 'du', 'syndrome'),
  ['NOM', 'ADJ', 'PRP', 'DET:ART', 'NOM', 'ADJ', 'PRP:det', 'NOM']],
 [('forme', 'inusitée'), ['NOM', 'ADJ']],
 [('forme', 'inusitée', 'du', 'syndrome'), ['NOM', 'ADJ', 'PRP:det', 'NOM']],
 [('forme', 'inusitée', 'du', 'syndrome', 'de', 'cushing'),
  ['NOM', 'ADJ', 'PRP:det', 'NOM', 'PRP', 'NOM']],
 [('forme', 'inusitée', 'du', 'syndrome', 'de', 'cushing', 'drs'),
  ['NOM', 'ADJ', 'PRP:det', 'NOM', 'PRP', 'NOM', 'ADJ']],
 [('syndrome', 'de', 'cushing'), ['NOM', 'PRP', 'NOM']],
 [('syndrome', 'de', 'cushing', 'drs'), ['NOM', 'PRP', 'NOM', 'ADJ']],
 [('syndrome', 'de', 'cushing', 'drs', 'andré', 'lacroix'),
  ['NOM', 'PRP', 'NOM', 'ADJ', 'ADJ', 'NOM']],
 [('cushing'

On retire les n-grammes qui apparaissent moins de 5 fois dans le corpus

In [34]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 5]

phrases = filter_freq(phrases)

In [35]:
phrases

[[('gène', 'responsable'), ['NOM', 'ADJ']],
 [('gène', 'responsable', "d'", 'une', 'forme'),
  ['NOM', 'ADJ', 'PRP', 'DET:ART', 'NOM']],
 [('gène', 'responsable', "d'", 'une', 'forme', 'inusitée'),
  ['NOM', 'ADJ', 'PRP', 'DET:ART', 'NOM', 'ADJ']],
 [('gène', 'responsable', "d'", 'une', 'forme', 'inusitée', 'du', 'syndrome'),
  ['NOM', 'ADJ', 'PRP', 'DET:ART', 'NOM', 'ADJ', 'PRP:det', 'NOM']],
 [('forme', 'inusitée'), ['NOM', 'ADJ']],
 [('forme', 'inusitée', 'du', 'syndrome'), ['NOM', 'ADJ', 'PRP:det', 'NOM']],
 [('forme', 'inusitée', 'du', 'syndrome', 'de', 'cushing'),
  ['NOM', 'ADJ', 'PRP:det', 'NOM', 'PRP', 'NOM']],
 [('syndrome', 'de', 'cushing'), ['NOM', 'PRP', 'NOM']],
 [('syndrome', 'de', 'cushing'), ['NOM', 'PRP', 'NOM']],
 [('glandes', 'surrénales'), ['NOM', 'ADJ']],
 [('étude', 'publiée'), ['NOM', 'VER:pper']],
 [('centre', 'de', 'recherche'), ['NOM', 'PRP', 'NOM']],
 [('centre', 'de', 'recherche', 'du', 'chum'),
  ['NOM', 'PRP', 'NOM', 'PRP:det', 'NOM']],
 [('centre', 'de',

In [36]:
#phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
phrases = [[term[0], " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [37]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

In [38]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 73345 occurrences de ngrammes.


In [39]:
phrases[:15]

[[('gène', 'responsable'), 'NOM ADJ', 6],
 [('gène', 'responsable', "d'", 'une', 'forme'), 'NOM ADJ PRP DET:ART NOM', 6],
 [('gène', 'responsable', "d'", 'une', 'forme', 'inusitée'),
  'NOM ADJ PRP DET:ART NOM ADJ',
  6],
 [('gène', 'responsable', "d'", 'une', 'forme', 'inusitée', 'du', 'syndrome'),
  'NOM ADJ PRP DET:ART NOM ADJ PRP:det NOM',
  6],
 [('forme', 'inusitée'), 'NOM ADJ', 6],
 [('forme', 'inusitée', 'du', 'syndrome'), 'NOM ADJ PRP:det NOM', 6],
 [('forme', 'inusitée', 'du', 'syndrome', 'de', 'cushing'),
  'NOM ADJ PRP:det NOM PRP NOM',
  6],
 [('syndrome', 'de', 'cushing'), 'NOM PRP NOM', 15],
 [('syndrome', 'de', 'cushing'), 'NOM PRP NOM', 15],
 [('glandes', 'surrénales'), 'NOM ADJ', 12],
 [('étude', 'publiée'), 'NOM VER:pper', 28],
 [('centre', 'de', 'recherche'), 'NOM PRP NOM', 1307],
 [('centre', 'de', 'recherche', 'du', 'chum'),
  'NOM PRP NOM PRP:det NOM',
  1116],
 [('centre', 'de', 'recherche', 'du', 'chum', '(', 'crchum', ')'),
  'NOM PRP NOM PRP:det NOM PUN NOM P

In [40]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

phrases

[[('gène', 'responsable'), 'NOM ADJ', 6],
 [('gène', 'responsable', "d'", 'une', 'forme'), 'NOM ADJ PRP DET:ART NOM', 6],
 [('gène', 'responsable', "d'", 'une', 'forme', 'inusitée'),
  'NOM ADJ PRP DET:ART NOM ADJ',
  6],
 [('gène', 'responsable', "d'", 'une', 'forme', 'inusitée', 'du', 'syndrome'),
  'NOM ADJ PRP DET:ART NOM ADJ PRP:det NOM',
  6],
 [('forme', 'inusitée'), 'NOM ADJ', 6],
 [('forme', 'inusitée', 'du', 'syndrome'), 'NOM ADJ PRP:det NOM', 6],
 [('forme', 'inusitée', 'du', 'syndrome', 'de', 'cushing'),
  'NOM ADJ PRP:det NOM PRP NOM',
  6],
 [('syndrome', 'de', 'cushing'), 'NOM PRP NOM', 15],
 [('syndrome', 'de', 'cushing'), 'NOM PRP NOM', 15],
 [('glandes', 'surrénales'), 'NOM ADJ', 12],
 [('étude', 'publiée'), 'NOM VER:pper', 28],
 [('centre', 'de', 'recherche'), 'NOM PRP NOM', 1307],
 [('centre', 'de', 'recherche', 'du', 'chum'),
  'NOM PRP NOM PRP:det NOM',
  1116],
 [('centre', 'de', 'recherche', 'du', 'chum', '(', 'crchum', ')'),
  'NOM PRP NOM PRP:det NOM PUN NOM P

In [41]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(DataFrame(phrases).drop_duplicates())))

Après filtrage, on a 2981 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [42]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:200] # Pour prendre les structures syntaxiques attestées dans les MeSH

In [43]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [44]:
terms = filter_patterns(phrases)
#terms = phrases

In [45]:
terms

[[('gène', 'responsable'), 'NOM ADJ', 6],
 [('gène', 'responsable', "d'", 'une', 'forme'), 'NOM ADJ PRP DET:ART NOM', 6],
 [('gène', 'responsable', "d'", 'une', 'forme', 'inusitée'),
  'NOM ADJ PRP DET:ART NOM ADJ',
  6],
 [('forme', 'inusitée'), 'NOM ADJ', 6],
 [('forme', 'inusitée', 'du', 'syndrome'), 'NOM ADJ PRP:det NOM', 6],
 [('forme', 'inusitée', 'du', 'syndrome', 'de', 'cushing'),
  'NOM ADJ PRP:det NOM PRP NOM',
  6],
 [('syndrome', 'de', 'cushing'), 'NOM PRP NOM', 15],
 [('syndrome', 'de', 'cushing'), 'NOM PRP NOM', 15],
 [('glandes', 'surrénales'), 'NOM ADJ', 12],
 [('étude', 'publiée'), 'NOM VER:pper', 28],
 [('centre', 'de', 'recherche'), 'NOM PRP NOM', 1307],
 [('centre', 'de', 'recherche', 'du', 'chum'),
  'NOM PRP NOM PRP:det NOM',
  1116],
 [('recherche', 'du', 'chum'), 'NOM PRP:det NOM', 1140],
 [('chum', '(', 'crchum', ')'), 'NOM PUN NOM PUN', 48],
 [('syndrome', 'de', 'cushing'), 'NOM PRP NOM', 15],
 [('syndrome', 'de', 'cushing'), 'NOM PRP NOM', 15],
 [('glandes', 

In [46]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 41 % des termes
On avait 73345 ngrammes, on en a maintenant 43297.


In [47]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [48]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [49]:
terms_patterns

[{'Expression': ('gène', 'responsable'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 6},
 {'Expression': ('gène', 'responsable', "d'", 'une', 'forme'),
  'Structure syntaxique': 'NOM ADJ PRP DET:ART NOM',
  'Fréquence': 6},
 {'Expression': ('gène', 'responsable', "d'", 'une', 'forme', 'inusitée'),
  'Structure syntaxique': 'NOM ADJ PRP DET:ART NOM ADJ',
  'Fréquence': 6},
 {'Expression': ('forme', 'inusitée'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 6},
 {'Expression': ('forme', 'inusitée', 'du', 'syndrome'),
  'Structure syntaxique': 'NOM ADJ PRP:det NOM',
  'Fréquence': 6},
 {'Expression': ('forme', 'inusitée', 'du', 'syndrome', 'de', 'cushing'),
  'Structure syntaxique': 'NOM ADJ PRP:det NOM PRP NOM',
  'Fréquence': 6},
 {'Expression': ('syndrome', 'de', 'cushing'),
  'Structure syntaxique': 'NOM PRP NOM',
  'Fréquence': 15},
 {'Expression': ('syndrome', 'de', 'cushing'),
  'Structure syntaxique': 'NOM PRP NOM',
  'Fréquence': 15},
 {'Expression': ('glandes', 'su

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [50]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [51]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 43297 ngrammes.


In [52]:
frequencies

FreqDist({('STOP',): 33147, ('de',): 25273, ('.',): 13057, ('et',): 9465, ('la',): 9425, ('le',): 9207, ('du',): 7517, ("l'",): 7145, ("d'",): 7080, ('des',): 6929, ...})

In [53]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [54]:
def llr_ngrammes(terms):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : fd_tokens[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = frequencies[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = frequencies[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom
                #if res == float('-inf') :
                #    res = 50000

                if p_value < 0.001 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr


In [55]:
significant_coll = llr_ngrammes(terms)
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on veut pas appliquer le LLR

In [56]:
significant_coll


[{'Terme': ('gène', 'responsable'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence (TF)': 6,
  'LLR': 55.134520234129965,
  'p-value': 1.1255663317375388e-13},
 {'Terme': ('gène', 'responsable', "d'", 'une', 'forme'),
  'Structure syntaxique': 'NOM ADJ PRP DET:ART NOM',
  'Fréquence (TF)': 6,
  'LLR': 104.28829757055627,
  'p-value': 1.7491909869756254e-24},
 {'Terme': ('gène', 'responsable', "d'", 'une', 'forme', 'inusitée'),
  'Structure syntaxique': 'NOM ADJ PRP DET:ART NOM ADJ',
  'Fréquence (TF)': 6,
  'LLR': 146.59383976511123,
  'p-value': 9.627658723120452e-34},
 {'Terme': ('forme', 'inusitée'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence (TF)': 6,
  'LLR': 104.28829757058352,
  'p-value': 1.7491909869515707e-24},
 {'Terme': ('forme', 'inusitée', 'du', 'syndrome'),
  'Structure syntaxique': 'NOM ADJ PRP:det NOM',
  'Fréquence (TF)': 6,
  'LLR': 105.11668598654494,
  'p-value': 1.1515115092807553e-24},
 {'Terme': ('forme', 'inusitée', 'du', 'syndrome', 'de', 'cushing'),

In [57]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len(terms) - len(significant_coll)) / len(terms) *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 164 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 0 % de nos n-grammes.


In [58]:
df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

# On veut faire join pour tous les termes[collocation]
def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df.to_csv(output_path)

df = df.drop(columns=['p-value'])

### **Filtrage - Fréquence documentaire**

In [59]:
dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}

In [60]:
max_df = round(0.95 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 95% des documents 
min_df = round(0.01 * nb_docs)         # Pour rejeter les termes qui se retrouvent dans moins de 1% des documents
max_df

1525

In [61]:
min_df

16

In [62]:
max_df_observed = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
max_df_observed

1152

In [63]:
max_df_to_remove = {term:df for term,df in dfs.items() if df > max_df}
max_df_to_remove

{}

In [64]:
dfs = {term:df for term,df in dfs.items() if df < max_df and df > min_df} # Si df = 0, c'est un artefact créé par le prétraitement lui-même

Porter attention aux termes qui sortent ici - en comptant la fréquence documentaire, techniquement on ne devrait pas retrouver de 0

In [65]:
zeros =  {term:df for term,df in dfs.items() if df == 0}  
print(len(zeros))
zeros 

0


{}

In [66]:
dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 

dfs

,Terme,Fréquence documentaire (DF)
0,centre de recherche,1152
1,recherche du chum,1077
2,centre de recherche du chum,1065
3,axe de recherche,959
4,chum axe,932
...,...,...
320,analyse de données,17
321,types de cancer,19
322,intérêts de recherche étude,24
323,chum professeure titulaire,21


In [67]:
df = df.merge(dfs, on='Terme').drop_duplicates()

df.sort_values(['Fréquence (TF)'], 
            axis=0,
            ascending=[False], 
            inplace=True)

In [68]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF)
0,centre de recherche,NOM PRP NOM,1307,-inf,1152
1,recherche du chum,NOM PRP:det NOM,1140,-inf,1077
2,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1116,-inf,1065
3,axe de recherche,NOM PRP NOM,961,-inf,959
4,chum axe,NOM NOM,932,-inf,932
...,...,...,...,...,...
320,analyse de données,NOM PRP NOM,16,177.865748,17
321,types de cancer,NOM PRP NOM,14,90.345462,19
322,intérêts de recherche étude,NOM PRP NOM NOM,12,40.035718,24
323,chum professeure titulaire,NOM ADJ ADJ,11,90.612989,21


In [69]:
list_terms = df['Terme'].tolist()

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [70]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
# kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

# ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'").replace("( ", "(").replace(" )", ")") for ng in ngrammes]

In [71]:
# ngrammes_kwic = [t for t in ngrammes_kwic if not 'STOP' in t]

In [72]:
# extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence (TF)'])
# kwic = {w : [] for w in kw} 

In [73]:
# for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
#     for w in kw:
#         if t.startswith(w):
#             kwic[w].append(t)

In [74]:
# kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [75]:
# for term in kw:
#     df_kw = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence (TF)"])
#     df_kw.sort_values(["Fréquence (TF)"], 
#         axis=0,
#         ascending=[False], 
#         inplace=True)

#     df_kw.insert(0, 'Mot-clé', term)
#     extrant = concat([extrant, df])


# extrant = extrant[extrant['Fréquence (TF)'] > 30] 

# # file_path = '../04-filtrage/output/'
# # file_path = path.join(file_path, acteur, tag)


# # extrant.to_csv(file_path + '_KWIC' +'.csv')

# extrant

### **Extraction de termes MeSH**

In [76]:
df['isMeSHTerm']= False # On set à False puis on va changer pour True si on trouve le terme
df['MeSHID'] = None
df['MesH_prefLabel_fr'] = None
df['MesH_prefLabel_en'] = None

In [77]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [78]:
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [79]:
for t in extr_mesh:
    if t in mesh:
        df.loc[df['Terme'] == t, 'isMeSHTerm'] = True
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF),isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
0,centre de recherche,NOM PRP NOM,1307,-inf,1152,False,None,None,None
1,recherche du chum,NOM PRP:det NOM,1140,-inf,1077,False,None,None,None
2,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1116,-inf,1065,False,None,None,None
3,axe de recherche,NOM PRP NOM,961,-inf,959,False,None,None,None
4,chum axe,NOM NOM,932,-inf,932,False,None,None,None
...,...,...,...,...,...,...,...,...,...
320,analyse de données,NOM PRP NOM,16,177.865748,17,True,None,None,None
321,types de cancer,NOM PRP NOM,14,90.345462,19,False,None,None,None
322,intérêts de recherche étude,NOM PRP NOM NOM,12,40.035718,24,False,None,None,None
323,chum professeure titulaire,NOM ADJ ADJ,11,90.612989,21,False,None,None,None


In [80]:
# Termes MeSH présents dans notre corpus 
df[df['isMeSHTerm'] == True]

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF),isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
51,médecine nucléaire,NOM ADJ,115,1368.721575,95,True,None,None,None
61,santé publique,NOM ADJ,104,1096.033310,70,True,None,None,None
65,instituts de recherche,NOM PRP NOM,101,818.775871,83,True,None,None,None
67,cancer de la prostate,NOM PRP DET:ART NOM,94,1362.401565,34,True,None,None,None
70,système immunitaire,NOM ADJ,93,1203.299177,49,True,None,None,None
89,santé mentale,NOM ADJ,74,796.919175,38,True,None,None,None
93,sclérose en plaques,NOM PRP NOM,71,1309.205180,25,True,None,None,None
96,soins infirmiers,NOM NOM,65,819.602180,36,True,None,None,None
102,maladie de parkinson,NOM PRP NOM,61,1003.752994,23,True,None,None,None
103,intelligence artificielle,NOM ADJ,60,1154.279483,33,True,None,None,None


### **Extraction de termes existant dans la taxonomie**

In [81]:
df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme

In [82]:
file_path = '../04-filtrage/default_taxo_labels.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    default = read_csv(f, sep=';')
    taxo_terms = list(dict.fromkeys([str(t).strip().lower() for t in default['Label'].tolist()]))

taxo_terms

['1 on 1 support',
 '10 weeks and less ultrasound',
 '12-step facilitation therapy',
 '12-step programs',
 '18f pet bone scan',
 '18f bone pet',
 '18f bone scan',
 '1st generation antipsychotics',
 '2019 ncov testing',
 '2d doppler echocardiography',
 '2d echocardiography',
 '2d ultrasound',
 '2nd generation antipsychotics',
 '3d imaging',
 '3d sonography',
 '3d dental imaging',
 '3d echocardiography',
 '3d fetal ultrasound',
 '3d mammography',
 '3d prenatal ultrasound',
 '3d reconstruction',
 '46, xx disorder of sex development',
 '46,xy disorder of sex development',
 '4d echocardiography',
 '4d prenatal ultrasound',
 'a1c diabetes test',
 'a1c test',
 'abo + rh pnl bld',
 'abo and rh group in cord blood',
 'abpm device',
 'acl reconstruction surgery',
 'acth deficiency',
 'acth stimulation test',
 'adhd assessment',
 'adhd diagnostic',
 'adhd evaluation',
 'adhd testing',
 'adn foetal circulant',
 'aids',
 'aids nephropathy',
 'aids serodiagnosis',
 'aids-associated nephropathy',
 'a

In [83]:
for t in df['Terme'].tolist():
    if t in taxo_terms:
        df.loc[df['Terme'] == t, 'isTaxoTerm'] = True
df[df['isTaxoTerm'] == True]

,Terme,Structure syntaxique,Fréquence (TF),LLR,Fréquence documentaire (DF),isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm
41,recherche clinique,NOM ADJ,147,430.529215,113,False,None,None,None,True
44,centre hospitalier,NOM ADJ,131,1334.341436,98,False,None,None,None,True
51,médecine nucléaire,NOM ADJ,115,1368.721575,95,True,None,None,None,True
61,santé publique,NOM ADJ,104,1096.033310,70,True,None,None,None,True
64,département de radiologie,NOM PRP NOM,101,1079.921280,87,False,None,None,None,True
67,cancer de la prostate,NOM PRP DET:ART NOM,94,1362.401565,34,True,None,None,None,True
70,système immunitaire,NOM ADJ,93,1203.299177,49,True,None,None,None,True
89,santé mentale,NOM ADJ,74,796.919175,38,True,None,None,None,True
93,sclérose en plaques,NOM PRP NOM,71,1309.205180,25,True,None,None,None,True
96,soins infirmiers,NOM NOM,65,819.602180,36,True,None,None,None,True


### **Mapping to MeSH IDs**

Lire le fichier XML contenant le MeSH Bilingue

Le MeSH 2019 comprend 29 351 descripteurs  
https://mesh.inserm.fr/FrenchMesh/presentation.htm

In [84]:
import xml.etree.ElementTree as ET
import re
import pandas as pd
base_path = '../04-filtrage/MeSH/'

tree = ET.parse(base_path + 'fredesc2019.xml')
root = tree.getroot()

def flatten(l):
    return [item for sublist in l for item in sublist]

data_mesh = [{'mesh_id' : x.find('DescriptorUI').text.strip('\n'), \
         'label_fr' : x.find('DescriptorName').find('String').text.split('[')[0], \
         'label_en' : x.find('DescriptorName').find('String').text.split('[')[1].strip(']'), \
         'synonymes (en/fr)' : flatten([[term.find('String').text for term in concept.find('TermList').findall('Term')] for concept in x.find('ConceptList').findall('Concept')]) \
         } for x in root.findall('DescriptorRecord')]

In [85]:
# for t in df[df['isMeSHTerm'] == True]['Terme'].tolist():
#     for d in data_mesh:
#         if t == str(d['label_fr']).lower():
#             df.loc[df['Terme'] == t, 'MeSHID'] = d['mesh_id']
#             df.loc[df['Terme'] == t, 'MesH_prefLabel_fr'] = d['label_fr']
#             df.loc[df['Terme'] == t, 'MesH_prefLabel_en'] = d['label_en']

In [86]:
for t in df[df['isMeSHTerm'] == True]['Terme'].tolist():
    for d in data_mesh:
        if t in [str(x).lower() for x in d['synonymes (en/fr)']]:
            df.loc[df['Terme'] == t, 'MeSHID'] = d['mesh_id']
            df.loc[df['Terme'] == t, 'MesH_prefLabel_fr'] = d['label_fr']
            df.loc[df['Terme'] == t, 'MesH_prefLabel_en'] = d['label_en']

df = df[['Terme', 'Structure syntaxique',	'Fréquence (TF)', 'Fréquence documentaire (DF)', 'LLR', 'isMeSHTerm', 'isTaxoTerm', 'MeSHID', 'MesH_prefLabel_fr', 'MesH_prefLabel_en']]

In [87]:
df[df['isMeSHTerm'] == True]

,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
51,médecine nucléaire,NOM ADJ,115,95,1368.721575,True,True,D009683,Médecine nucléaire,Nuclear Medicine
61,santé publique,NOM ADJ,104,70,1096.033310,True,True,D011634,Santé publique,Public Health
65,instituts de recherche,NOM PRP NOM,101,83,818.775871,True,False,D000047,Académies et instituts,Academies and Institutes
67,cancer de la prostate,NOM PRP DET:ART NOM,94,34,1362.401565,True,True,D011471,Tumeurs de la prostate,Prostatic Neoplasms
70,système immunitaire,NOM ADJ,93,49,1203.299177,True,True,D007107,Système immunitaire,Immune System
89,santé mentale,NOM ADJ,74,38,796.919175,True,True,D008603,Santé mentale,Mental Health
93,sclérose en plaques,NOM PRP NOM,71,25,1309.205180,True,True,D009103,Sclérose en plaques,Multiple Sclerosis
96,soins infirmiers,NOM NOM,65,36,819.602180,True,True,D009732,Soins infirmiers,Nursing Care
102,maladie de parkinson,NOM PRP NOM,61,23,1003.752994,True,True,D010300,Maladie de Parkinson,Parkinson Disease
103,intelligence artificielle,NOM ADJ,60,33,1154.279483,True,False,D001185,Intelligence artificielle,Artificial Intelligence


### **Mapping to WikiData IDs**

In [88]:
from qwikidata.sparql import (get_subclasses_of_item, return_sparql_query_results)

In [89]:
df.insert(len(df.columns), 'WikiDataID', None)
df

,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,WikiDataID
0,centre de recherche,NOM PRP NOM,1307,1152,-inf,False,False,None,None,None,None
1,recherche du chum,NOM PRP:det NOM,1140,1077,-inf,False,False,None,None,None,None
2,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1116,1065,-inf,False,False,None,None,None,None
3,axe de recherche,NOM PRP NOM,961,959,-inf,False,False,None,None,None,None
4,chum axe,NOM NOM,932,932,-inf,False,False,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
320,analyse de données,NOM PRP NOM,16,17,177.865748,True,False,D000078332,Analyse de données,Data Analysis,None
321,types de cancer,NOM PRP NOM,14,19,90.345462,False,False,None,None,None,None
322,intérêts de recherche étude,NOM PRP NOM NOM,12,24,40.035718,False,False,None,None,None,None
323,chum professeure titulaire,NOM ADJ ADJ,11,21,90.612989,False,False,None,None,None,None


In [90]:
no_match = []
for t in df['Terme'].tolist():
    # On va compter le nombree de termes pour lesquels on ne trouve pas de QID  

    ## Requête SPARQL
    #Prend une liste de preflabels@en en minuscules sans virgule (convention de wikidata)
    #Retourne les meshID si trouvés et le prefLabel sous "term"
    #itemLabel = PrefLabel de Wikidata
    query = """
        SELECT ?item ?itemLabel ?term WHERE {{   
        VALUES ?term {{
            "{}"@fr
        }}  
        ?item rdfs:label|skos:altLabel ?term;
                
        SERVICE wikibase:label {{bd:serviceParam wikibase:language "fr".}} }}
    """.format(t)
    
    try:
        res = return_sparql_query_results(query)
        if(len(res['results']['bindings'])>0):
            try:
                uri = return_sparql_query_results(query)['results']['bindings'][0]['item']['value']
                print(uri)
                qid = re.search('Q\d+', uri).group(0)
                print(qid)

                # Consigner le QID dans le dataframe 
                df.loc[df['Terme'] == t, 'WikiDataID'] = qid

            except :
                no_match.append(t)
    except:
        no_match.append(t)

http://www.wikidata.org/entity/Q7315155
Q7315155


In [91]:
df.insert(0, 'Corpus', acteur)
df

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,WikiDataID
0,chum,centre de recherche,NOM PRP NOM,1307,1152,-inf,False,False,None,None,None,Q7315155
1,chum,recherche du chum,NOM PRP:det NOM,1140,1077,-inf,False,False,None,None,None,None
2,chum,centre de recherche du chum,NOM PRP NOM PRP:det NOM,1116,1065,-inf,False,False,None,None,None,None
3,chum,axe de recherche,NOM PRP NOM,961,959,-inf,False,False,None,None,None,None
4,chum,chum axe,NOM NOM,932,932,-inf,False,False,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
320,chum,analyse de données,NOM PRP NOM,16,17,177.865748,True,False,D000078332,Analyse de données,Data Analysis,None
321,chum,types de cancer,NOM PRP NOM,14,19,90.345462,False,False,None,None,None,None
322,chum,intérêts de recherche étude,NOM PRP NOM NOM,12,24,40.035718,False,False,None,None,None,None
323,chum,chum professeure titulaire,NOM ADJ ADJ,11,21,90.612989,False,False,None,None,None,None


In [92]:
if lng == 'en':
    acteur = acteur + '_' + lng

output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df.to_csv(output_path)

In [93]:
df[df['isMeSHTerm'] == True]

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,WikiDataID
51,chum,médecine nucléaire,NOM ADJ,115,95,1368.721575,True,True,D009683,Médecine nucléaire,Nuclear Medicine,None
61,chum,santé publique,NOM ADJ,104,70,1096.033310,True,True,D011634,Santé publique,Public Health,None
65,chum,instituts de recherche,NOM PRP NOM,101,83,818.775871,True,False,D000047,Académies et instituts,Academies and Institutes,None
67,chum,cancer de la prostate,NOM PRP DET:ART NOM,94,34,1362.401565,True,True,D011471,Tumeurs de la prostate,Prostatic Neoplasms,None
70,chum,système immunitaire,NOM ADJ,93,49,1203.299177,True,True,D007107,Système immunitaire,Immune System,None
89,chum,santé mentale,NOM ADJ,74,38,796.919175,True,True,D008603,Santé mentale,Mental Health,None
93,chum,sclérose en plaques,NOM PRP NOM,71,25,1309.205180,True,True,D009103,Sclérose en plaques,Multiple Sclerosis,None
96,chum,soins infirmiers,NOM NOM,65,36,819.602180,True,True,D009732,Soins infirmiers,Nursing Care,None
102,chum,maladie de parkinson,NOM PRP NOM,61,23,1003.752994,True,True,D010300,Maladie de Parkinson,Parkinson Disease,None
103,chum,intelligence artificielle,NOM ADJ,60,33,1154.279483,True,False,D001185,Intelligence artificielle,Artificial Intelligence,None
